## Video Load

In [ ]:
!pip install opencv-python
!pip install matplotlib==3.3.0

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            cv2.imshow("video", frame)
            cv2.waitKey(ms) # Millisecond 단위로 대기
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [ ]:
# Video가 저장된 경로 입력
PATH ='/Users/yunjinkim/Desktop/디지털영상처리/video.mov'
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [5]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
print(f"- Maximum Intensity: {video.max()}")
print(f"- Minimum Intensity: {video.min()}")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (4181, 442, 978, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0


In [7]:
import cv2
import numpy as np

def load_video(path, ms=100):
    capture = cv2.VideoCapture(path)
    frames = []
    
    while capture.isOpened():
        run, frame = capture.read()
        if run:
            frames.append(frame)
        else:
            break
        cv2.imshow("video", frame)
        cv2.waitKey(ms)
    capture.release()
    cv2.destroyAllWindows()dnld
    return np.array(frames, dtype='uint8')

def preprocess_frame(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 피부색 범위 설정 (손의 피부색 감지)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # 피부색 영역 검출
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # 블러링 적용 (노이즈 제거)
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    
    return blurred

def detect_hand_area(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_position_and_bounding_box(contours):
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        position = (x + w // 2, y + h // 2)  # 중앙 위치
        return position, (x, y, w, h)
    return None, None

def draw_box(frame, box, color=(255, 0, 0)):
    if box:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

def count_pushups(frames):
    pushup_count = 0
    initial_hand_position = None
    is_touching = False

    for frame in frames:
        mask = preprocess_frame(frame)
        
        hand_contours = detect_hand_area(mask)

        hand_position, hand_box = get_position_and_bounding_box(hand_contours)

        draw_box(frame, hand_box, (255, 0, 0))

        if hand_position:
            current_hand_y = hand_position[1]  # 손의 y 위치

            if initial_hand_position is None:
                initial_hand_position = current_hand_y

            # 손의 색상이 변하면 (무언가 닿으면) 푸쉬업 카운트
            if current_hand_y < initial_hand_position - 5 and not is_touching:
                is_touching = True
            elif current_hand_y > initial_hand_position + 5 and is_touching:
                is_touching = False
                pushup_count += 1

            initial_hand_position = current_hand_y

        cv2.imshow("Processed Video", frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    return pushup_count

# 비디오 경로 설정
video_path = '/Users/yunjinkim/Desktop/디지털영상처리/video.mov'

# 비디오 로드
video_frames = load_video(video_path)

# 푸쉬업 카운팅
pushup_count = count_pushups(video_frames)
print(f"Total Push-ups Counted: {pushup_count}")




Total Push-ups Counted: 95


In [ ]:
import cv2
import numpy as np

def calculate_circularity(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * area) / (perimeter ** 2)
    return circularity

def find_largest_circularity_contour(contours):
    max_circularity = -1
    largest_contour = None
    
    for contour in contours:
        circularity = calculate_circularity(contour)
        if circularity > max_circularity:
            max_circularity = circularity
            largest_contour = contour
    
    return largest_contour

def find_center_of_contour(contour):
    ((x, y), _radius) = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    return center

def high_boost_filter(image, kernel_size=3, k=1.2):
    blur = cv2.blur(image, (kernel_size, kernel_size))
    high_boost_image = image * k - blur
    return high_boost_image

def preprocess_frame(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    intensity = hsv[:, :, 2]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(intensity, cv2.MORPH_DILATE, kernel)
    hsv[:, :, 2] = opening
    result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return result

def detect_skin(img):
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255)) 
    HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135)) 
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
    global_mask = cv2.medianBlur(global_mask, 5)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((7,7), np.uint8))
    return global_mask

def detect_hand_area(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_position_and_bounding_box(contours):
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        position = (x + w // 2, y + h // 2)
        return position, (x, y, w, h)
    return None, None

def draw_box(frame, box, color=(255, 0, 0)):
    if box:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

def count_pushups(head_positions):
    count = 0
    state = 0  # 0: 시작, 1: 내려감, 2: 올라옴
    for i in range(1, len(head_positions)):
        if head_positions[i-1] is None or head_positions[i] is None:
            continue
        if state == 0 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
        elif state == 1 and head_positions[i][1] < head_positions[i-1][1]:
            count += 1
            state = 2
        elif state == 2 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
    return count

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    head_positions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        preprocessed_frame = preprocess_frame(frame)
        skin_mask = detect_skin(preprocessed_frame)
        hand_contours = detect_hand_area(skin_mask)
        hand_position, hand_box = get_position_and_bounding_box(hand_contours)
        
        if hand_position:
            head_positions.append(hand_position)
            draw_box(frame, hand_box)

        cv2.imshow("Processed Video", frame)
        
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    pushup_count = count_pushups(head_positions)
    print(f"Total Push-ups Counted: {pushup_count}")

if __name__ == "__main__":
    video_path = "pushup_sample2.mp4"
    main(video_path)


In [ ]:
#64회